In [4]:
#Crawl 
import requests
import re
from bs4 import BeautifulSoup
from newspaper import Article
import urllib.request

# Crawl HTML

In [2]:
req = ['https://www.thegioididong.com/dtdd?p=tu-4-7-trieu',
       'https://www.thegioididong.com/dtdd?p=tu-7-13-trieu',
       'https://www.thegioididong.com/dtdd?p=tren-13-trieu',
]
def crawlPhoneLinks(req):
    phoneLinks = set()
    for r in req:
        r1 = requests.get(r)
        coverpage = r1.content
        soup1 = BeautifulSoup(coverpage, 'html.parser')
        for a in soup1.find_all('a', href=True):
            if re.search('/dtdd/.*', a['href']):
                if a.find_all('div', class_='ratingresult'):
                    phoneLinks.add(a['href'])
        print(len(phoneLinks))
    return phoneLinks
phoneLinks = crawlPhoneLinks(req)

def crawlRevLink(phoneLinks):
    reviewLinks=set()
    for link in phoneLinks:
        if link.count('https://www.thegioididong.com')==0:
            link = 'https://www.thegioididong.com'+link
        link = link +'/danh-gia?p='
        reviewLinks.add(link)
        #print(link)
    print(len(reviewLinks))
    return reviewLinks
reviewLinks = crawlRevLink(phoneLinks)

def crawlRev(reviewLinks):
    reviews = []
    for link in reviewLinks:
        i = 1
        while True:
            link = link+str(i)
            try:
                page = urllib.request.urlopen(link)
                soup = BeautifulSoup(page, 'html.parser')
                new_feeds = soup.find(
                    'ul', class_='ratingLst').find_all('li')

                article = Article(link)
                article.download()
                article.parse()
                soup = BeautifulSoup(article.html, 'html.parser')
                html = soup.findAll("div", {"class": "rc"})
                reviews = reviews + html

            except:
                break;
            i = i+1
    #print(len(reviews))
    return reviews
reviews = crawlRev(reviewLinks)

20
36
56
56


In [18]:
comments = []
stars = []
labels = []
for li in reviews:
        rev = str(li).split('\n')
        if len(rev)>9:
            comment = rev[9].replace('<i>','').replace('</i>','')
            if comments.count(comment)==0:
                comments.append(comment)
                star = str(li).count('txtstar')
                stars.append(star)
                if star>3:
                    labels.append(1)
                else: 
                    labels.append(0)
print(len(comments))

2016


In [19]:
# process 
import re
import pyvi
from pyvi import ViTokenizer #For split vietnamese words

In [20]:
# ================== process raw data ===========

REPLACE_NO_SPACE = re.compile("[.;:\',\"()\[\]][.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    reviews = [line.lstrip() for line in reviews]
    reviews = [ViTokenizer.tokenize(rev) for rev in reviews]
    return reviews
comments_clean = preprocess_reviews(comments)

In [21]:
with open('TxtData/dataComment.txt','w',encoding="utf8") as f:
    f.write('\n'.join(comments_clean))
    f.close()
with open('TxtData/dataLabel.txt','w',encoding="utf8") as f:
    f.write(str(labels[0]))
    for l in labels[1:len(labels)]:
        f.write('\n'+str(l))
    f.close()

In [22]:
#======================
zipped_pairs = zip(labels,comments_clean ) 
comments_clean = [x for _, x in sorted(zipped_pairs)] 
labels = sorted(labels)

an_iterator = filter(lambda number: number ==0, labels)
numNeg = len(list(an_iterator))
print(numNeg)

with open('TxtData/data.txt','w',encoding="utf8") as f:
    f.write(str(numNeg))
    f.write('\n'.join(comments_clean))
    f.close()

737
